This is the beginning of the Toronto clustering and analysis project.  Below, the data is scraped from the Wikepedia page.

In [1]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests

The following cells are the first task of Unit 3 of Course 9 of the series. This scrapes the html file off the website, then loops on the table, identified in the html by the collection of 'tr's.  Each tr for data has 3 td's in it, the Postal Code, the Borough and the Neighborhood, in that order.  These are assigned to a list during each cycle and the list populates a dictionary.  The dictionary is used to create a dataframe, which, in turn, is appended to the Toronto dataframe, tordf.  The responses to 'Not assigned' are also included in the loop. That completes the process of creating the basic dataframe.

In [2]:
html_file=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(html_file, 'lxml')
table=soup.find('table')
#print (table)
tordf=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'],dtype=str)
ent=['a','b','c']
pdlib={'Postalcode':'a','Borough':'b','Neighborhood':'c'}
j=1
for tr in table.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        ent[i]=td.text
        i=i+1
    if ent[1]=='Not assigned':
        continue;
    if ent[0]=='a':
        continue;
    ent[2]=ent[2].replace('\n',"")
    if ent[2]=='Not assigned':
        ent[2]=ent[1]
    pdlib['Neighborhood']=ent[2]
    pdlib['Postalcode']=ent[0]
    pdlib['Borough']=ent[1]
    tordf=tordf.append([pdlib],ignore_index=True)
tordf.head(10)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now, to eliminate Postal Code duplication by combining neighborhoods in one Postal Code row. A dataframe is created using Groupby, grouping on Postal Code and Borough together. Commas are inserted into the Neighborhood entries via an .apply to the join of the data. A reset_index is used to make the index of the original groupby become the first two columns, i.e. Postalcode and Borough. The original Groupby sorted the Postal Codes.

In [3]:
tordf2=pd.DataFrame(tordf.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join),columns=['Neighborhood']).reset_index()
tordf2.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
tordf2.shape

(103, 3)

103 Postal Codes, each with the Borough and the list of Neighborhoods.

This completes the first part of the assignment for Unit 3 of Course 9.

The following is the second part of the assignment for Unit 3 of Course 9. The next step in the project is to add longitude and latitude columns to the tordf2 df. Looping through the Postal Codes will be necessary, creating a pair of lists.  These lists are then assigned to new columns in tordf2.

In [5]:
!pip install geocoder
import geocoder # import geocoder


    100% |████████████████████████████████| 102kB 7.3MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

I never got the geocoder to work.  The loop sought eternally.  So, on to the csv file.  That will require loading it, sorting it, then finding the latitude and longitude for the Postal Code.

In [6]:
dset=pd.read_csv("http://cocl.us/Geospatial_data")
print(dset.head(10))
dset.shape

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
5         M1J  43.744734 -79.239476
6         M1K  43.727929 -79.262029
7         M1L  43.711112 -79.284577
8         M1M  43.716316 -79.239476
9         M1N  43.692657 -79.264848


(103, 3)

The latitude and longitude list is in alphabetical order by Postal Code, just like tordf2.  The length is also the same; all the Postal Codes are included. One only needs to copy the latitude and longitude columns onto the tordf2 dataframe.

In [7]:
tordf2['Latitude']=dset['Latitude']
tordf2['Longitude']=dset['Longitude']
tordf2.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(tordf2['Borough'].unique()),
        tordf2.shape[0]
    )
)

The dataframe has 11 boroughs and 103 postal codes.


This concludes the 2nd section of the assignment for Unit 3 of Course 9.